In [1]:
# Listing 7.17 - The standard workflow using `compile()`, `fit()`, `evaluate()`, `predict()`
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = layers.Dense(512, activation="relu")(inputs) 
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60_000, 28 * 28)).astype("float32") / 32
test_images = test_images.reshape((10_000, 28 * 28)).astype("float32") / 32
train_images, val_images = images[10_000:], images[:10_000]
train_labels, val_labels = labels[10_000:], labels[:10_000]

model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels,
          epochs=3,
          validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

Epoch 1/3
1563/1563 [==============================] - 4s 2ms/step - loss: 0.4060 - accuracy: 0.8997 - val_loss: 0.1868 - val_accuracy: 0.9590
Epoch 2/3
1563/1563 [==============================] - 3s 2ms/step - loss: 0.2594 - accuracy: 0.9448 - val_loss: 0.1617 - val_accuracy: 0.9645
Epoch 3/3
313/313 [==============================] - 0s 547us/step


In [2]:
# Listing 7.18 - Implementing a custom metric by subclassing the `Metric` class
import tensorflow as tf

class RootMeanSquaredError(keras.metrics.Metric):

    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

In [3]:
# using the custom metric
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy", RootMeanSquaredError()])
model.fit(train_images, train_labels,
          epochs=3,
          validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

Epoch 1/3
1563/1563 [==============================] - 4s 2ms/step - loss: 0.4110 - accuracy: 0.8992 - rmse: 7.3119 - val_loss: 0.1854 - val_accuracy: 0.9585 - val_rmse: 7.4219
Epoch 2/3
1563/1563 [==============================] - 3s 2ms/step - loss: 0.2537 - accuracy: 0.9460 - rmse: 7.4002 - val_loss: 0.1660 - val_accuracy: 0.9661 - val_rmse: 7.4328
Epoch 3/3
313/313 [==============================] - 0s 554us/step
